Grading for pre-lecture questions is all or nothing. Partial credit is available for in-class assignments and checkpoints. Partial credit will primarily be determined by the quality of the comments you make in your code. Tell me what you intend the code to do. If there are no comments, there is no partial credit!

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# Learning Objectives

This lecture will show you how to:
1. Apply the relaxation method to time-independent PDEs
2. Setup and use the FTCS method for time-dependent PDEs
3. Use implicit methods to avoid numerical instability that is inherent to some PDEs

In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

import grading_helper as _test

# Relaxation Method for PDEs

In [ ]:
%video c4LwNoh8ZBM

Summary:

- The relaxation method is more commonly used with PDEs than with ODEs.
- We replace each derivative with a finite difference and work with a 2-D or 3-D array, but otherwise the method looks the same as the 1-D ODE case.

## Your Turn

Poisson's equation is similar to Laplace's equation, except that there is nonzero charge density in the region of interest.

$$\nabla^2V=-\frac{\rho}{\epsilon_0}\,,$$

where $\rho$ is the charge density and $\epsilon_0$ is the electrostatic constant.

Working in units where $\epsilon_0=1$, find $V$ everywhere within a $101\times101$ square region (where h=0.01) that has a uniform charge density of $\rho=1$ for the central $41\times41$ grid points. The walls of the $101\times101$ square region are grounded ($V=0$). Store the result in a 2-D array named `V` and plot `V` using `imshow`.

In [ ]:
%%graded # 2 points

# boundary conditions
V = np.zeros((101, 101))
rho = np.zeros((101, 101))
rho[30:-30, 30:-30] = 1 # inner 41x41

# YOUR CODE HERE

In [ ]:
%%tests

_test.equal(V[:,0], 0)  # boundary conditions
_test.equal(V[:,-1], 0) #
_test.equal(V[0,:], 0)  #
_test.equal(V[-1:0], 0) #
assert V.max() == V[50,50] # max value should be at center
_test.similar(V[50,50], 0.0337) # check center
_test.plot_shown()

# FTCS Method

In [ ]:
%video w1Uudu1HeN4

Summary:

- Forward-Time Centered-Space. Used for PDEs with both time and space as independent variables.
- Pairs a finite difference for space with Euler's method for time.

**Recipe**
1. Write spacial part of PDE as finite difference. Result is a set of simultaneous ODEs (treat each grid point as a unique dependent variable).
3. Use forward Euler method to solve time dependence.

## Your Turn

In 2-D, the diffusion equation takes the form

$$\frac{\partial T}{\partial t}= D\,\nabla^2T= D\,\bigg(\frac{\partial^2T}{\partial x^2} + \frac{\partial^2T}{\partial y^2}\bigg)\,.$$

A tall steel rod with square $10\times10$ cm cross-section at an initial uniform temperature of 100$^\circ$C is immersed in a bath of 0$^\circ$C water. Use `imshow` to plot the internal temperature of the steel through a cross-section at time $t=10$ s after being immersed in the water. Assume that the temperature change of the water is negligible during this time. The thermal diffusivity for steel is $D=4\times10^{-6}$ m$^2$/s. Store the temperature in a 2-D array named `T`.

In [ ]:
%%graded # 3 points

D, L = 4e-6, 0.1

time = np.linspace(0, 10, 1001)
h = 1e-2

T = 100*np.ones((101,101)) # initial conditions
a = 1e-3

T[0, :] = T[-1, :] = T[:, 0] = T[:, -1] = 0 # boundary conditions

for t in time:
    # apply FTCS method, and insert result here
    # YOUR CODE HERE
    
plt.imshow(T, cmap="hot", extent=(0, 10, 0, 10), vmin=0, vmax=100)
plt.colorbar()
plt.show()

In [ ]:
%%tests

_test.equal(T[:,0], 0)  # boundary conditions
_test.equal(T[:,-1], 0) #
_test.equal(T[0,:], 0)  #
_test.equal(T[-1:0], 0) #
_test.similar(T[51, 51], 100) # middle still hot
_test.similar(T[51, :5], [0, 8.90, 17.7, 26.3, 34.5]) # check surface layers
_test.plot_shown()

# Crank-Nicolson Method

In [ ]:
%video oXx6q6cxORk

Summary:

- The FTCS method is numerically unstable for some PDEs.
- The solution is to use the Crank-Nicolson method, which combines the forward and backward Euler methods:

 $$\frac{y(x, t+h) - y(x,t)}{h}=\tfrac12\big[f(x, t) + f(x, t+h)\big]\,.$$
 
- Unfortunately, this aproach makes things harder since we will loose our explicit expression for $y(x, t+h)$. The result is a system of equations.
- At each time step, you need to solve a tridiagonal matrix. `solve_banded` works well here.

# Additional Resources

- Textbook Sections 9.1 - 9.3. There is a detailed treatment of the issue of numerical stability in section 9.3.2